In [1]:
using SDDP, Distributions, GLPK, Plots

In [2]:
graph = SDDP.Graph(
           :root_node,
           [:decision_node,:demand_node],
           [
               (:root_node => :decision_node, 1.0),
               (:decision_node => :demand_node, 1.0)
           ]);


p = 1.0 #day ahead price
q = 2.0 #same day price

model = SDDP.PolicyGraph(
            graph,
            sense = :Min,
            lower_bound = 0,
            optimizer = with_optimizer(GLPK.Optimizer)) do subproblem, node

                @variable(subproblem, stock>=0, SDDP.State, initial_value = 0)

                @variable(subproblem, 0 <= reserve)
                @variable(subproblem, 0 <= shortage)

                @variable(subproblem, demand)

                @constraint(subproblem, stock.out == stock.in + reserve + shortage - demand)

                if node == :decision_node
                        JuMP.fix(demand,0.0)
                        JuMP.fix(shortage,0.0;force=true)
                else

                    ##demandas uniformes
                    demand_range = collect(0:100);
                    demand_probability = ones(size(demand_range));
                    demand_probability ./= sum(demand_probability);

                    SDDP.parameterize(subproblem,demand_range,demand_probability) do d
                        JuMP.fix(demand,d)
                        JuMP.fix(reserve,0.0;force=true)
                    end
                end


                if node == :decision_node
                    @stageobjective(subproblem,  p*reserve);
                else
                    @stageobjective(subproblem,  q*shortage);
                end
            end

A policy graph with 2 nodes.
 Node indices: decision_node, demand_node


In [6]:
#SDDP.train(model,iteration_limit=20)
#AVAR version
SDDP.train(model,risk_measure = SDDP.AVaR(0.2),iteration_limit=20)

--------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-20

Numerical stability report
  Non-zero Matrix range     [1e+00, 2e+00]
  Non-zero Objective range  [1e+00, 2e+00]
  Non-zero Bounds range     [0e+00, 0e+00]
  Non-zero RHS range        [7e+01, 1e+02]
No problems detected

Solver: serial mode

 Iteration    Simulation       Bound         Time (s)    Proc. ID   # Solves
        1    1.080000e+02   9.039604e+01   3.322079e-01          1       4266
        2    9.039604e+01   9.539992e+01   3.347750e-01          1       4370
        3    8.539216e+01   9.542574e+01   3.370121e-01          1       4474
        4    8.800000e+01   9.544059e+01   3.394649e-01          1       4578
        5    8.950000e+01   9.544554e+01   3.429880e-01          1       4682
        6    9.000000e+01   9.544554e+01   3.458021e-01          1       4786
        7    9.000000e+01   9.544554e+01   3.481500e-01          1       48

In [7]:
results=SDDP.simulate(model,1,[:reserve,:stock,:demand,:shortage])

1-element Array{Array{Dict{Symbol,Any},1},1}:
 [Dict(:stock => SDDP.State{Float64}(0.0, 89.99999999999991),:bellman_term => 5.445544554455537,:noise_term => nothing,:node_index => :decision_node,:stage_objective => 89.99999999999991,:objective_state => nothing,:reserve => 89.99999999999991,:demand => 0.0,:belief => Dict(:decision_node => 1.0),:shortage => 0.0…), Dict(:stock => SDDP.State{Float64}(89.99999999999991, 49.999999999999915),:bellman_term => 0.0,:noise_term => 40,:node_index => :demand_node,:stage_objective => 0.0,:objective_state => nothing,:reserve => 0.0,:demand => 40.0,:belief => Dict(:demand_node => 1.0),:shortage => 0.0…)]